In [5]:
import pandas as pd
import geopandas as gpd
import folium
import branca
import json
from shapely.geometry import Polygon, MultiPolygon, Point
import matplotlib.pyplot as plt
import io
import requests

In [58]:
#GEODATA ONLINE
df_deptos = gpd.read_file('https://www.dropbox.com/s/y30ylt8qaz516yu/MGN_DPTO_POLITICO.geojson?dl=1',driver='GeoJSON')
s = requests.get("https://www.dropbox.com/s/0wu5t06rfuyyebo/mintic_investments_clean.csv?dl=1").content
df_investment = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [59]:
#order_info by department
df_investment=df_investment.sort_values(by=['departame_cod'],ascending=True)
df_investment_total=df_investment.groupby(['anio_corte','departame_cod','viceministerio'])['inversion'].agg('sum').reset_index()
df_investment_total.head(15)

,anio_corte,departame_cod,viceministerio,inversion
0,1990,11,transformacion,0
1,1990,13,transformacion,0
2,1990,15,transformacion,0
3,1990,17,transformacion,0
4,1990,18,transformacion,0
5,1990,19,transformacion,0
6,1990,20,transformacion,0
7,1990,23,transformacion,0
8,1990,25,transformacion,0
9,1990,41,transformacion,0


In [61]:
#CLEAN DATA
df_investment_total.loc[df_investment_total['departame_cod'].str.len()==1,'departame_cod'] = '0' + df_investment_total['departame_cod'].astype(str)
#MERGE DATA
df_merge = pd.merge(df_investment_total, df_deptos,  how='left', left_on=['departame_cod'], right_on = ['DPTO_CCDGO'])

In [63]:
df_merge.dropna()
df_merge.sort_values(by=['anio_corte','viceministerio'],ascending=False)

,anio_corte,departame_cod,viceministerio,inversion,DPTO_CCDGO,DPTO_CNMBR,DPTO_ANO_C,DPTO_ACT_A,DPTO_NAREA,DPTO_CSMBL,DPTO_VGNC,SHAPE_AREA,SHAPE_LEN,geometry
645,2021,00,transformacion,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
647,2021,11,transformacion,300000000,11,"BOGOTÁ, D.C.",1538.0,Constitucion Politica de 1886,1622.852605,3,2020.0,0.132208,3.760453,"POLYGON ((-74.15067 4.00485, -74.15062 4.00500..."
649,2021,13,transformacion,402000000,13,BOLÍVAR,1886.0,Constitucion Politica de 1886,26719.211406,3,2020.0,2.195577,16.234817,"MULTIPOLYGON (((-75.22221 10.79660, -75.22222 ..."
651,2021,15,transformacion,911200000,15,BOYACÁ,1886.0,Constitucion Politica de 1886,23138.048132,3,2020.0,1.888391,15.906491,"POLYGON ((-72.04767 7.03830, -72.04761 7.03818..."
653,2021,17,transformacion,434000000,17,CALDAS,1905.0,11 de Abril de 1905,7425.221672,3,2020.0,0.605498,6.655844,"POLYGON ((-74.67054 5.76896, -74.67154 5.77147..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,1990,08,transformacion,0,08,ATLÁNTICO,1910.0,Ley 21 de 1910,3313.810147,3,2020.0,0.273770,2.544651,"POLYGON ((-74.91077 10.25418, -74.91027 10.254..."
22,1990,85,transformacion,0,85,CASANARE,1991.0,5 de Julio Constitucion Politica de 1991,44394.239774,3,2020.0,3.615063,12.132754,"POLYGON ((-69.83591 6.04126, -69.83591 6.04126..."
23,1990,86,transformacion,0,86,PUTUMAYO,1991.0,Articulo 309 Constitucion Politica de 1991,25976.283105,3,2020.0,2.107965,12.707922,"POLYGON ((-76.08495 1.05755, -76.08545 1.05771..."
24,1990,88,transformacion,0,88,"ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANT...",1991.0,Artículo 310 Constitucion Politica de 1991,49.424251,3,2020.0,0.004050,0.685259,"MULTIPOLYGON (((-81.70420 12.59481, -81.70427 ..."


In [64]:
#FUNCTION TO GET DATA FROM DEPARMENT, YEAR, AND VICEMINISTRY
def geo_per_year_vice(d,year,vice):
    df=pd.DataFrame(data=d)
    df=df.loc[(df['anio_corte'] == year) & (df['viceministerio'] == vice)]
    return df 

In [65]:
df_merge=geo_per_year_vice(df_merge,2021,'conectividad')

In [66]:
## GET MAX MIN VALUE INVESTMENT PER VICEMINISTRY PER YEAR
column = df_merge["inversion"]
max_value = column.max()
#print(max_value)
min_value = column.min()
#print(min_value)

144716244486
3480150168


In [67]:
#style specification
colormap = branca.colormap.LinearColormap(
    colors=['white','yellow','orange','red','darkred'],
    vmin=min_value, # min_value 0
    vmax=max_value # max_value 35
)

In [68]:
style_function = lambda x: {
    'fillColor': colormap(x['properties']['INVESTMENT']), #VARIABLE SHOULD BE INVESTMENT
    'color': 'black',
    'weight':2,
    'fillOpacity':0.5
}

In [72]:
def iterator(object,cod,nom,var,var1):
    for obj in object:
        for attribute, value in obj.items():
            if "properties" in attribute:
                obj['properties']={"DPTO_CCDGO": cod, "DPTO_CNMBR": nom, "INVESTMENT": var, "VICEMINISTRY":var1} #VARIABLE SHOULD BE INVESTMENT
    return object

In [75]:
map = folium.Map(location=[4.714331, -74.112808], zoom_start=5 , tiles='CartoDB positron')#tiles="Stamen Terrain"
#REMOVE INVALID DATA 
df_merge = df_merge[df_merge.geometry != None]

for _, r in df_merge.iterrows():  #df_deptos.iterrows() ITERATE df_merge
    #without simplifying the representation of each borough, the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    t = json.loads(geo_j)
    t['features']=iterator(t['features'],r['DPTO_CCDGO'],r['DPTO_CNMBR'],r['inversion'],r['viceministerio']) #VARIABLE SHOULD BE INVESTMENT
    geo_j=t
    
    #dep=geo_j['features'][0]['properties']['DPTO_CNMBR']
    #var=geo_j['features'][0]['properties']['VARIABLE']
    
    geo_j = folium.GeoJson(data=geo_j,style_function=style_function,tooltip=folium.GeoJsonTooltip(
                           fields=['DPTO_CNMBR','INVESTMENT','VICEMINISTRY'],
                           aliases=['Department', 'Invesment','Viceministry'], 
                           localize=True)
            )  
    #folium.Popup(r['DPTO_CNMBR']).add_to(geo_j)
    geo_j.add_to(map)
    #STILL STATIC
map.save("data/conectividad2021.html")    
map

In [56]:
map